# Tutorial

This notebook provides a basic tutorial on how to implement three basic tasks using the [ArDCA](https://github.com/pagnani/ArDCAData) package in [Julia](https://julialang.org/). We will assume we have a Multiple Sequence Alignment (MSA)in FASTA format. We aim at

1. Given a MSA, generate a sample
2. Given a MSA, predict contacts
3. Given a MSA, predict the mutational effect in all (ungapped) position of a given target sequence 


## Installing the software

1. Install [julia](https://julialang.org/downloads/)

2. Install the ipynb interface in julia, you should add the [IJulia](https://github.com/JuliaLang/IJulia.jl) package using the package manager. Again, from the package manager prompt do a
```
(@v1.5) pkg> add IJulia 
```
Exit the Package Manager (type `backspace` or `delete`) and from julia prompt do a 
```
julia> using IJulia
julia> notebook()
```
The ipynb file manager should open a page on your browser and from there you should navigate to this notebook.


## Load ArDCA package 

The following cell loads the package `ArDCA` (*Warning*: the first time it takes a while). To fully exploit the the multicore parallel computation, julia should be invoked with

```
$ julia -t nthreads # put here nthreads equal to the number of cores you want to use
```

If you want to set permanently the number of threads to the desired value, you can either create a default environment variable `export JULIA_NUM_THREADS=24` in your `.bashrc`. More information [here](https://docs.julialang.org/en/v1.6/manual/multi-threading/)

The `mypkgdir` variable should be set to your `path/to/package` dir.

In this notebook we will use the PF00014 protein family available in `data/PF14/` folder of the package/


In [2]:
mypkgdir = normpath(joinpath(pwd(),".."))
datadir=joinpath(mypkgdir,"data") # put here your path
using Pkg
Pkg.activate(mypkgdir)
using ArDCA

## Learn the autoregressive parameters

  Activating environment at `~/CODE/ArDCA/Project.toml`


## 0. Learning the ArDCA parameters

As a preliminary step, we learn the field and the coupling parameters $h,J$ from the MSA. To do so we use the `ardca` method that return the parameters (stored in `arnet` in the cell below), and the alignment in numerical format and other algorithms variables (stored in `arvar` in the cell below). The default autoregressive order is set to `:ENTROPIC`. We set the $L_2$ regularization to 0.02 for the $J$ and 0.001 for the $h$.

The keywork arguments for the `ardca` method are (with their default value):

* `epsconv::Real=1.0e-5` (convergenge parameter)

* `maxit::Int=1000` (maximum number of iteration - don't change)

* `verbose::Bool=true` (set to `false` to suppress printing on screen)

* `method::Symbol=:LD_LBFGS` (optimization method)

* `permorder::Union{Symbol,Vector{Ti}}=:ENTROPIC` (permutation order). Possible values are: `[:NATURAL, :ENTROPIC, :REV_ENTROPIC, :RANDOM]` or a custom permutation vector.




In [3]:
fastafile = joinpath(datadir,"PF14/PF00014_mgap6.fasta.gz")
arnet,arvar=ardca(fastafile, verbose=false, lambdaJ=0.02,lambdaH=0.001);

removing duplicate sequences... done: 13600 -> 8871
θ = 0.3308846126401105 threshold = 17.0
M = 8871 N = 53 Meff = 2950.925530761646


## 1. Sequence Generation

We now generate `M` sequences using the `sample` method:

In [4]:
M = 1_000;
generated_alignment = sample(arnet,M);

The generated alignment has is a $L\times M$ matrix of integer where $L$ is the alignment's length, and $M$ the number of samples.

Interestingly, we for each sequence we can also compute the likelihood with the `sample_with_weights` method.

In [5]:
loglikelihood,generated_alignment = sample_with_weights(arnet,M);

## 2. Contact Prediction

We can compute the epistatic score for residue-residue contact prediction. To do so, we can use the `epistatic_score` method. The epistatic score is computed on any target sequence of the MSA. Empirically, it turns out the the final score does not depend much on the choice of the target sequence. 

The autput is contained in a `Vector` of `Tuple` ranked in descendic score order. Each `Tuple` contains $i,j,s_{ij}$ where $s_{ij}$ is the epistatic score of the residue pair $i,j$. The residue numbering is that of the MSA, and not of the unaligned full sequences.

In [6]:
target_sequence = 1
score=epistatic_score(arnet,arvar,target_sequence);

## 3. Predicting mutational effects

For any reference sequence, we can easily predict the mutational effect for all single mutants. Of course we can extract this information only for the *non-gapped* residues of the target sequence. 

This is done with the `dms_single_site` method, which returns a `q×L` matrix `D` containing $\log(P(mut))/\log(P(wt))$ for all single
site mutants of the reference sequence `seqid` (i.e. the so-called wild type sequence), and `idxgap` a vector of indices of the residues of the reference sequence that contain gaps (i.e. the 21
amino-acid) for which the score has no sense and is set by convention to `+Inf`.

A negative value indicate a beneficial mutation, a value 0 indicate
the wild-type amino-acid.

In [8]:
target_sequence = 1
D,idxgap=dms_single_site(arnet,arvar,target_sequence);